In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# GraphRAG on Google Cloud (Part Two)
## Vertex AI Agent Framework



|Author(s) | [Smitha Venkat](https://github.com/smitha-google), [Tristan Li](https://github.com/codingphun)


## Overview

In the [first part](https://github.com/codingphun/gcp_graph_rag/blob/main/graph_rag_spanner.ipynb) of the tutorial, GraphRAG on Google Cloud, you learned how to use popular open-source LangChain framework. In this second part tutorial, you will learn how to use Vertex AI Agent Framework. The goal Vertex AI Agent Framework is to deliver a industry’s leading, enterprise ready agent platform that can power real world agentic use cases, along with state-of-the-art exemplar solutions built on top of the Google Cloud platform.

## Objectives

In this tutorial, you will leverage the graph and vector database created in part one tutorial and build an agent with the Vertex AI Agent Framework. You will also learn how to use Function Calling in the agent to look up information in knowledge graph stored in Spanner. Finally, you will learn how to leverage Gemini Google Search grounding to look up additional information that may not exist in the knowledge graph.


## Before you begin

1. Make sure to [sign up](https://docs.google.com/forms/d/1uTPDw0TWrj7EZRXE-8UTl4mpsTWelBjDu3H-0FFWlXg/viewform?edit_requested=true) for the Vertex AI Agent Framework first to receive the SDK needed for this tutorial.
1. Make sure to run the [first part](https://github.com/codingphun/gcp_graph_rag/blob/main/graph_rag_spanner.ipynb) of the tutorial to create the knowledge graph, embedding vectors in the Spanner database.
1. In the Google Cloud console, on the project selector page, select or [create a Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects).
1. [Make sure that billing is enabled for your Google Cloud project](https://cloud.google.com/billing/docs/how-to/verify-billing-enabled#console).


### Required roles

To get the permissions that you need to complete the tutorial, ask your administrator to grant you the [Owner](https://cloud.google.com/iam/docs/understanding-roles#owner) (`roles/owner`) IAM role on your project. 
For more information about granting roles, see [Manage access](https://cloud.google.com/iam/docs/granting-changing-revoking-access).


## Getting Started

### Install Python Libraries

In [1]:
# Install the genai agent whl file in the current instance.

!pip3 install google_genai_agents-0.0.2.dev*.whl --quiet --ignore-requires-python

!pip install google-cloud-spanner==2.0.0 --quiet
!pip install --upgrade google-cloud-aiplatform --quiet

### Restart the Kernel

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
print(sys.version)
# If using local jupyter instance, uncomment and run:
# !gcloud auth login

### CHANGE the following settings

In [24]:
import os

GOOGLE_CLOUD_PROJECT = "" 
LOCATION = "us-central1"
SPANNER_INSTANCE_ID='graphrag-instance'
SPANNER_DATABASE_ID='graphrag'
MODEL_NAME='gemini-1.5-flash-002'
EMBEDDING_MODEL_NAME='text-embedding-004'

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

### Import packages

In [25]:
#Import the packages

from agents.tools.base_tool import ToolContext
from google.cloud import spanner
from agents import Runner, Agent
import vertexai
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
from vertexai.generative_models import (
    Tool,
    grounding,
    GenerativeModel,
    GenerationConfig,
)

### Create helper functions for function calling

We will create a couple python functions for function calling for the agent. The first function is to query the Spanner database for database through semantic search and graph traversal. The second function is to leverage Gemini Google Seach grounding to look for additional information that may not exist in the Spanner knowledge graph.

In [54]:
# Helper Methods
def get_embedding(text, task_type, model):
  """Gets embeddings for the input text.

  Args:
    response: text, task_type, model

  Returns:
    The values from the embedding model
  """
  try:
    text_embedding_input = TextEmbeddingInput(task_type=task_type, text=text)
    embeddings = model.get_embeddings([text_embedding_input])
    return embeddings[0].values
  except:
    return []

def ask_graph(query:str) -> list[dict]:
  """Asks the pre-built graph database. We are using Cloud Spanner as our Graph Database for this example.
  Refer https://github.com/codingphun/gcp_graph_rag/blob/main/graph_rag_spanner.ipynb on how to build your Graph Database.

  Args:
    query: The question to ask the graph database.

  Returns:
    Output from the graph database
  """
  # Set the Spanner Database credentials
  spanner_client = spanner.Client(GOOGLE_CLOUD_PROJECT)
  instance = spanner_client.instance(SPANNER_INSTANCE_ID)
  database = instance.database(SPANNER_DATABASE_ID)
  TASK_TYPE = "QUESTION_ANSWERING"
  EMBEDDING_MODEL = TextEmbeddingModel.from_pretrained(EMBEDDING_MODEL_NAME)

  # Get the embeddings for the user query.
  q_emb = get_embedding(query, TASK_TYPE, EMBEDDING_MODEL)

  # Now the graph nodes, relationship and embedding vectors are stored in the Spanner database.
  kgnodename=''
  with database.snapshot() as snapshot:
    results = snapshot.execute_sql(
      """SELECT DocId, NAME, Doc FROM KgNode ORDER BY COSINE_DISTANCE(DocEmbedding, @q_emb) limit 1""",
      params={"q_emb": q_emb},
      param_types={"q_emb": spanner.param_types.Array(spanner.param_types.FLOAT64)}  # Adjust FLOAT64 if needed
      )
    for row in results:
      kgnodename=str(row[1])

    # Traverse the Knowledge Graph after you get the node to uncover the relationships and connections.
    tables = ["NATIONALITY", "WORKED_AT", "INVESTMENT", "INFLUENCED_BY"]
    a = []
    for table in tables:
       with database.snapshot() as snapshot:
        if table in ("INFLUENCED_BY", "LOCATED_IN"):
          column_name = "P1_Name"
        else:
          column_name = "P_Name"

        results = snapshot.execute_sql(
            f"""
            SELECT {column_name} AS Name, *
            FROM {table}
            WHERE {column_name} = '{kgnodename}'
            """
        )
        for row in results:
            # Dynamically determine the index of the relevant column
            name_index = row.index(kgnodename)
            # Construct the output string using f-string
            output_string = f"{row[name_index]} {table} {row[2]}"
            print(output_string)
  return f"""[{output_string}]"""

def ask_gemini(question: str) -> str:
  """Ask Gemini the question grounded with Google Search.

  Args:
    question: The question to ask Gemini.

  Returns:
    The answer from Gemini models grounded with Google Search.
  """
  model = GenerativeModel(MODEL_NAME)
  # Use Google Search for grounding
  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())
  prompt = f"""Please answer the question:{question}
  Use markdown to structure your answer if it makes sense. Be comprehensive in your answer.
  """
  response = model.generate_content(
      prompt,
      tools=[tool],
      generation_config=GenerationConfig(
          temperature=0.0,
      ),
  )
  return response.text

### Create an Agent

We will create a Vertex AI Agent and give some high-level instructions for the agent on how to interact with the users. Note the function calling directions in the instruction section.

In [41]:
# Defining the agent using the agent framework
root_agent = Agent(
    model=MODEL_NAME,
    name='root_agent',
    greeting_prompt = 'Tell the user about yourself and ask for the query.',
    instruction = """ - You are a helpful information retrieval agent that can answer user's query from the knowledge graph
                      and do a broader search if you cant find answer in the graph database.
                    - After you get the user query, always check the graph database first.
                    - If the query can be answered from the graph, then call the ask_graph tool.
                    - If you are not able to find the answer in the graph, ask the user if they would like to do a broader search.
                    - If the user says yes, then call the ask_gemini tool.
                    - If the user says no, then ask them if there is anything else they would like to know.
                    - Always be courteous and dont assume anything.
                    - If you dont know an answer, please say I dont know the answer.
                """,
    flow='auto',
    tools=[ask_graph, ask_gemini],
)

We will create a session for the agent to run in

In [42]:
# Session Management
from agents.sessions.in_memory_session_service import InMemorySessionService
from agents.artifacts.in_memory_artifact_service import InMemoryArtifactService
from google.genai import types


session_service = InMemorySessionService()
artifact_service = InMemoryArtifactService()
runner = Runner(root_agent, artifact_service, session_service)
session = session_service.create()

def run_prompt(new_message: str):
    content = types.Content(role='user', parts=[types.Part.from_text(text=new_message)])
    print (content)
    for event in runner.run(
      session=session,
      new_message=content,
    ):
        if event.content:
          print(event.content.model_dump(exclude_none=True))

Let's test out the agent

In [43]:
# Test your Agent
run_prompt('Who influenced Larry Page')

parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='Who influenced Larry Page')] role='user'
{'parts': [{'function_call': {'args': {'query': 'Who influenced Larry Page?'}, 'name': 'ask_graph'}}], 'role': 'model'}
Lawrence Edward Page WORKED_AT Alphabet Inc.
Lawrence Edward Page WORKED_AT Google
Lawrence Edward Page INVESTMENT Kitty Hawk
Lawrence Edward Page INVESTMENT Opener
Lawrence Edward Page INFLUENCED_BY Maria Montessori
{'parts': [{'function_response': {'name': 'ask_graph', 'response': {'result': '[Lawrence Edward Page INFLUENCED_BY Maria Montessori]'}}}], 'role': 'user'}
{'parts': [{'text': 'Based on the graph database, Maria Montessori influenced Larry Page.\n'}], 'role': 'model'}


### Google Searching Grounding

Currently, the agent retrieves answers from the Spanner-backed knowledge graph.  For queries beyond the knowledge graph's scope, we can augement it with Google Search.

In [51]:
# Calling the agent with another question that cannot be answered from knowledge graph
run_prompt('What is deep seek')

parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='What is deep seek')] role='user'
{'parts': [{'function_call': {'args': {'query': 'What is deep seek?'}, 'name': 'ask_graph'}}], 'role': 'model'}
Lawrence Edward Page WORKED_AT Alphabet Inc.
Lawrence Edward Page WORKED_AT Google
Lawrence Edward Page INVESTMENT Kitty Hawk
Lawrence Edward Page INVESTMENT Opener
Lawrence Edward Page INFLUENCED_BY Maria Montessori
{'parts': [{'function_response': {'name': 'ask_graph', 'response': {'result': '[Lawrence Edward Page INFLUENCED_BY Maria Montessori]'}}}], 'role': 'user'}
{'parts': [{'text': 'I am sorry, I could not find the answer to your question in the graph database. Would you like me to perform a broader search using another tool?\n'}], 'role': 'model'}


In [52]:
# Answer yes to ask your agent to perform a Google Search
run_prompt("Yes")

parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='Yes')] role='user'
{'parts': [{'function_call': {'args': {'question': 'What is deep seek?'}, 'name': 'ask_gemini'}}], 'role': 'model'}
{'parts': [{'function_response': {'name': 'ask_gemini', 'response': {'result': 'DeepSeek is a Chinese artificial intelligence (AI) company founded in May 2023 by Liang Wenfeng, who also co-founded the Chinese quantitative hedge fund High-Flyer, which owns DeepSeek.  The company is based in Hangzhou, Zhejiang, China.\n\nDeepSeek\'s primary focus is the development of open-source large language models (LLMs).  Their flagship model, DeepSeek-R1, is notable for achieving performance comparable to models like OpenAI\'s GPT-4, but at a significantly lower cost (reportedly $6 million versus $100 million for GPT-4) and using considerably less computing power (one-tenth).  This cost-e

### Troubleshooting

The agent framework provides a robust list of each reasoning step taken by the agent which makes troubleshooting easy

In [ ]:
# List out active sessions
session_service.list_sessions()

Make sure you replace the "session_id" placeholder below from the above step, if there are multiple sessions returned, use the last one

In [49]:
session_service.get("8ce93f23-1c76-4887-a0a2-5b5fc046bf60").model_dump()["events"]

[{'invocation_id': 'vMW81019',
  'author': 'user',
  'content': {'parts': [{'video_metadata': None,
     'thought': None,
     'code_execution_result': None,
     'executable_code': None,
     'file_data': None,
     'function_call': None,
     'function_response': None,
     'inline_data': None,
     'text': 'Who influenced Larry Page'}],
   'role': 'user'},
  'actions': {'skip_summarization': None,
   'state_delta': None,
   'artifact_delta': None,
   'pending': None,
   'transfer_to_agent': None,
   'escalate': None},
  'is_greeting': None,
  'function_call_event_id': None,
  'partial': None,
  'code_execution_event_type': None,
  'id': 'UhNTqv1C',
  'timestamp': 1738337945.694343},
 {'invocation_id': 'vMW81019',
  'author': 'root_agent',
  'content': {'parts': [{'video_metadata': None,
     'thought': None,
     'code_execution_result': None,
     'executable_code': None,
     'file_data': None,
     'function_call': {'id': None,
      'args': {'query': 'Who influenced Larry Page?'

### Clean Up

*   Delete the Spanner instance


In [ ]:
!gcloud spanner instances delete {SPANNER_INSTANCE_ID} --quiet